In [ ]:
# Install GLPK solver (if not already installed)
!sudo apt-get install glpk-utils
!which glpsol

# Import necessary libraries
from pyomo.environ import *
import numpy as np
import random
import math
import logging
from pyomo.util.infeasible import log_infeasible_constraints
logging.basicConfig(level=logging.INFO)
from pyomo.util.infeasible import log_infeasible_constraints, log_infeasible_bounds

# ====================== DATA DEFINITION ======================
# Define sets and parameters
N = ['p1', 'p2', 'p3', 'd1', 'd2', 'd3', 'o1', 'o2', 't1']
D = ['o1', 'o2']
T = ['t1']
R = ['r1', 'r2', 'r3']
V = ['v1', 'v2']

pickup_nodes = {'r1': 'p1', 'r2': 'p2', 'r3': 'p3'}
delivery_nodes = {'r1': 'd1', 'r2': 'd2', 'r3': 'd3'}

c = {('p1', 'd1'): 10, ('p2', 'd2'): 15, ('p3', 'd3'): 20,
     ('p1', 't1'): 5, ('t1', 'd1'): 5, ('p2', 't1'): 10, ('t1', 'd2'): 10,
     ('p3', 't1'): 15, ('t1', 'd3'): 15, ('o1', 'p1'): 5, ('o2', 'p2'): 5,
     ('d1', 'o1'): 5, ('d2', 'o2'): 5}

t = {('p1', 'd1'): 5, ('p2', 'd2'): 10, ('p3', 'd3'): 15,
     ('p1', 't1'): 2, ('t1', 'd1'): 2, ('p2', 't1'): 5, ('t1', 'd2'): 5,
     ('p3', 't1'): 7, ('t1', 'd3'): 7, ('o1', 'p1'): 3, ('o2', 'p2'): 3,
     ('d1', 'o1'): 3, ('d2', 'o2'): 3}

Q = {'v1': 20, 'v2': 25}
B = {'v1': 300, 'v2': 350}
H = {'v1': 5, 'v2': 6}
L = {'p1': 1, 'p2': 2, 'p3': 3, 'd1': -1, 'd2': -2, 'd3': -3, 'o1': 0, 'o2': 0, 't1': 0}
E = {'p1': 0, 'p2': 0, 'p3': 0, 'd1': 0, 'd2': 0, 'd3': 0, 'o1': 0, 'o2': 0, 't1': 0}
T_i = {'p1': 2000, 'p2': 2000, 'p3': 2000, 'd1': 2000, 'd2': 2000, 'd3': 2000, 'o1': 2000, 'o2': 2000, 't1': 5000}
C_i_values = {'t1': 1}
M = 100000
# Define the model
model = ConcreteModel()
# Define sets
model.N = Set(initialize=N)
model.D = Set(initialize=D)
model.T = Set(initialize=T)
model.R = Set(initialize=R)
model.V = Set(initialize=V)

# Define parameters
model.c = Param(model.N, model.N, initialize=c, default=0)
model.t = Param(model.N, model.N, initialize=t, default=0)
model.Q = Param(model.V, initialize=Q)
model.B = Param(model.V, initialize=B)
model.H = Param(model.V, initialize=H)
model.L = Param(model.N, initialize=L)
model.E = Param(model.N, initialize=E)
model.T_i = Param(model.N, initialize=T_i)
model.M = Param(initialize=M)
model.C_i = Param(model.T, initialize=C_i_values, default=0)
model.t_charge = Param(initialize=10)  # Charge time per transfer node

# Define decision variables
model.x = Var(model.N, model.N, model.V, within=Binary, initialize=0)  # Yij equivalent
model.y = Var(model.N, model.N, model.V, model.R, within=Binary, initialize=0)  # Xijk equivalent
model.z = Var(model.T, model.V, model.V, model.R, within=Binary, initialize=0)
model.p = Var(model.T, model.V, within=NonNegativeReals, initialize=0)
model.a = Var(model.N, model.V, within=NonNegativeReals, initialize=0)
model.d = Var(model.N, model.V, within=NonNegativeReals, initialize=0)
model.b = Var(model.N, model.V, within=NonNegativeReals, initialize=0)
model.W_k = Var(model.V, within=NonNegativeReals, initialize=1)  # Initialize waiting time cost to 1
model.q_i = Var(model.T, within=NonNegativeIntegers, initialize=0)  # Number of vehicles waiting at transfer node i
model.q_aux = Var(model.T, within=Reals, initialize=0)
model.s = Var(model.T, within=NonNegativeReals, initialize=0)  # Slack variable
model.slack_transfer = Var(model.V, model.V, model.T, within=NonNegativeReals, bounds=(0, M))
# Objective function
def objective_rule(model):
    return sum(model.c[i, j] * model.x[i, j, k] for i in model.N for j in model.N if i != j for k in model.V) + \
           sum(model.W_k[k] for k in model.V) + 1  # Include waiting time cost and a small positive constant
model.objective = Objective(rule=objective_rule, sense=minimize)
# Constraints
# Equation (2): Each vehicle leaves its origin depot
def leave_depot_rule(model, k):
    return sum(model.x[i, j, k] for i in model.N for j in model.N if i != j and j == list(model.D)[0]) == 1
model.leave_depot = Constraint(model.V, rule=leave_depot_rule)

# Equation (3): Each vehicle returns to its destination depot
def return_depot_rule(model, k):
    return sum(model.x[i, j, k] for i in model.N for j in model.N if i != j and i == list(model.D)[1]) == 1
model.return_depot = Constraint(model.V, rule=return_depot_rule)

# Constraint 4: If a vehicle enters a node, it must leave the node
def flow_rule(model, k, i):
    if i not in model.D:  # Ensure that the node is not a depot
        return sum(model.x[i, j, k] for j in model.N if j != i) == sum(model.x[j, i, k] for j in model.N if j != i)
    else:
        return Constraint.Skip
model.flow_constraint = Constraint(model.V, model.N, rule=flow_rule)

# Constraint 5: Each request must be picked up from its pickup node
def pickup_rule(model, r):
    return sum(model.y[pickup_nodes[r], delivery_nodes[r], k, r] for k in model.V) == 1
model.pickup_constraint = Constraint(model.R, rule=pickup_rule)


# Constraint 6: Each request must be delivered to its delivery node
def delivery_rule(model, r):
    return sum(model.y[i, j, k, r] for k in model.V for i in model.N for j in model.N if i != j and j == delivery_nodes[r]) == 1
model.delivery_constraint = Constraint(model.R, rule=delivery_rule)

# Constraint 7: If a request passes through a transfer node, it must leave the transfer node
def transfer_rule(model, r, i):
    if i in model.T:
        return sum(model.y[i, j, k, r] for k in model.V for j in model.N if j != i) == sum(model.y[j, i, k, r] for k in model.V for j in model.N if j != i)
    else:
        return Constraint.Skip
model.transfer_constraint = Constraint(model.R, model.N, rule=transfer_rule)

# Constraint 8: If a request enters a node that is not pickup, delivery, or transfer, it must leave with the same vehicle
def same_vehicle_rule(model, r, i):
    if i not in model.T and i not in model.D and i not in pickup_nodes.values() and i not in delivery_nodes.values():
        return sum(model.y[i, j, k, r] for k in model.V for j in model.N if j != i) == sum(model.y[j, i, k, r] for k in model.V for j in model.N if j != i)
    else:
        return Constraint.Skip
model.same_vehicle_constraint = Constraint(model.R, model.N, rule=same_vehicle_rule)

# Constraint 9: If a vehicle does not travel from node i to j, it cannot carry a request through arc (i, j)
def connection_rule(model, k, r, i, j):
    if i != j:
        return model.y[i, j, k, r] <= model.x[i, j, k]
    else:
        return Constraint.Skip
model.connection_constraint = Constraint(model.V, model.R, model.N, model.N, rule=connection_rule)

# Constraint 10: Relationship between departure and arrival times
def time_relation_rule(model, k, i, j):
    if i != j:
        return model.d[i, k] + model.t[i, j] - model.a[j, k] <= model.M * (1 - model.x[i, j, k])
    else:
        return Constraint.Skip
model.time_relation_constraint = Constraint(model.V, model.N, model.N, rule=time_relation_rule)

# Constraint 11: Departure time cannot be earlier than arrival time
def departure_after_arrival_rule(model, k, i):
    if i not in model.T:
        return model.a[i, k] <= model.d[i, k]
    else:
        return Constraint.Skip
model.departure_after_arrival_constraint = Constraint(model.V, model.N, rule=departure_after_arrival_rule)

# Constraint 12: Departure time from transfer nodes includes charging time
def charging_time_rule(model, k, i):
    if i in model.T:
        return model.a[i, k] + model.p[i, k] <= model.d[i, k]
    else:
        return Constraint.Skip
model.charging_time_constraint = Constraint(model.V, model.N, rule=charging_time_rule)

# Constraint 13: Arrival time at pickup nodes must be within the time window
def pickup_time_window_rule(model, k, r):
    pickup_node = pickup_nodes[r]
    return model.a[pickup_node, k] >= model.E[pickup_node]
model.pickup_time_window_constraint = Constraint(model.V, model.R, rule=pickup_time_window_rule)

# Constraint 14: Departure time from delivery nodes must be within the time window
def delivery_time_window_rule(model, k, r):
    delivery_node = delivery_nodes[r]
    return model.a[delivery_node, k] <= model.T_i[delivery_node]
model.delivery_time_window_constraint = Constraint(model.V, model.R, rule=delivery_time_window_rule)
# Constraint 15: Ensure that the z variable is 1 if a request is transferred between vehicles at a transfer node
def transfer_decision_rule(model, i, k, l, r):
    if i in model.T and k != l:
        return sum(model.y[i, j, k, r] for j in model.N if j != i) + \
               sum(model.y[j, i, l, r] for j in model.N if j != i) <= 1 + model.z[i, k, l, r]
    else:
        return Constraint.Skip
model.transfer_decision_constraint = Constraint(model.T, model.V, model.V, model.R, rule=transfer_decision_rule)


# Constraint 16: Arrival time of pickup vehicle must be earlier than departure time of delivery vehicle
def transfer_time_rule(model, i, k, l, r):
    if k != l:
        return model.a[i, k] - model.d[i, l] <= model.M * (1 - model.z[i, k, l, r])
    else:
        return Constraint.Skip
model.transfer_time_constraint = Constraint(model.T, model.V, model.V, model.R, rule=transfer_time_rule)

# Constraint 17: Vehicle capacity constraint
def capacity_rule(model, k, i, j):
    if i != j:
        return sum(model.L[i] * model.y[i, j, k, r] for r in model.R) <= model.Q[k] * model.x[i, j, k]
    else:
        return Constraint.Skip
model.capacity_constraint = Constraint(model.V, model.N, model.N, rule=capacity_rule)

# Constraint 18: Battery level calculation after leaving regular nodes
def battery_regular_rule(model, k, i, j):
    if i not in model.T and j != 0:
        return model.b[j, k] <= model.b[i, k] - model.t[i, j] + model.M * (1 - model.x[i, j, k])
    else:
        return Constraint.Skip
model.battery_regular_constraint = Constraint(model.V, model.N, model.N, rule=battery_regular_rule)

# Constraint 19: Battery level calculation after leaving transfer nodes
def battery_transfer_rule(model, k, i, j):
    if i in model.T and j != 0:
        return model.b[j, k] <= model.b[i, k] - model.t[i, j] + model.M * (1 - model.x[i, j, k]) + model.H[k] * model.p[i, k]
    else:
        return Constraint.Skip
model.battery_transfer_constraint = Constraint(model.V, model.N, model.N, rule=battery_transfer_rule)

# Constraint 20: Vehicles leave the depot with a full battery
def initial_battery_rule(model, k):
    return model.b[model.D.at(1), k] == model.B[k]
model.initial_battery_constraint = Constraint(model.V, rule=initial_battery_rule)

# Constraint 21: Vehicles cannot be charged beyond their battery capacity
def max_charging_rule(model, k, i):
    if i in model.T:
        # Ensure that the battery level after charging does not exceed the battery capacity
        return model.b[i, k] + model.H[k] * model.p[i, k] <= model.B[k]
    else:
        return Constraint.Skip
# Constraint 22: Vehicles must visit a transfer node to be recharged
def charging_node_rule(model, k, i):
    if i in model.T:
        return model.p[i, k] <= (model.B[k] / model.H[k]) * sum(model.x[j, i, k] for j in model.N)
    else:
        return Constraint.Skip
model.charging_node_constraint = Constraint(model.V, model.N, rule=charging_node_rule)

# Constraint 23: Precedence constraint (pickup before delivery)
def precedence_rule(model, r, k, i, j):
    if i == pickup_nodes[r] and j == delivery_nodes[r]:
        return model.a[i, k] <= model.a[j, k]
    else:
        return Constraint.Skip
model.precedence_constraint = Constraint(model.R, model.V, model.N, model.N, rule=precedence_rule)

# Constraint 24: No vehicle can enter an origin depot except at the start of the route
def no_enter_origin_depot_rule(model, k, i):
    if i in model.D:
        return sum(model.x[j, i, k] for j in model.N if j != i) <= 4
    else:
        return Constraint.Skip

# Constraint 25: No vehicle can leave a destination depot except at the end of the route
def no_leave_destination_depot_rule(model, k, i):
    if i in model.D:
        return sum(model.x[i, j, k] for j in model.N if j != i) <= 4
    else:
        return Constraint.Skip

# Constraint 26: No vehicle can travel from a delivery node to its associated pickup node
def no_delivery_to_pickup_rule(model, r, k, i, j):
    if i == delivery_nodes[r] and j == pickup_nodes[r]:
        return model.x[i, j, k] == 0
    else:
        return Constraint.Skip

# Constraint 27: No vehicle can travel between nodes if the distance exceeds battery capacity
def battery_distance_rule(model, k, i, j):
    if i != j and model.t[i, j] > model.B[k]:
        return model.x[i, j, k] == 0
    else:
        return Constraint.Skip
model.battery_distance_constraint = Constraint(model.V, model.N, model.N, rule=battery_distance_rule)

# Constraint 28: No vehicle can enter a transfer node unless it is recharging
def no_enter_transfer_without_recharge_rule(model, k, i):
    if i in model.T:
        return sum(model.x[j, i, k] for j in model.N if j != i) <= sum(model.p[i, k] for k in model.V)
    else:
        return Constraint.Skip
model.no_enter_transfer_without_recharge = Constraint(model.V, model.N, rule=no_enter_transfer_without_recharge_rule)

# Constraint 29: No vehicle can leave a transfer node unless it is recharged
def no_leave_transfer_without_recharge_rule(model, k, i):
    if i in model.T:
        return sum(model.x[i, j, k] for j in model.N if j != i) <= sum(model.p[i, k] for k in model.V)
    else:
        return Constraint.Skip
model.no_leave_transfer_without_recharge = Constraint(model.V, model.N, rule=no_leave_transfer_without_recharge_rule)

# Constraint 30: No vehicle can travel in a loop (i.e., from i to j and back to i)
def no_loop_rule(model, k, i, j):
    if i != j:
        return model.x[i, j, k] + model.x[j, i, k] <= 1
    else:
        return Constraint.Skip
model.no_loop_constraint = Constraint(model.V, model.N, model.N, rule=no_loop_rule)

# Constraint 31: Order matching constraint for pickup and delivery nodes
def order_matching_rule(model, k, r1, r2, i, j, m):
    if i == pickup_nodes[r1] and j == delivery_nodes[r1] and m == delivery_nodes[r2]:
        return model.x[i, j, k] + model.x[j, m, k] + model.x[i, m, k] <= 2
    else:
        return Constraint.Skip
model.order_matching_constraint = Constraint(model.V, model.R, model.R, model.N, model.N, model.N, rule=order_matching_rule)


# New Constraints from the provided content
# Constraint 32: Charger capacity constraints
def charger_capacity_rule(model, i):
    return sum(model.x[i, j, k] for j in model.N for k in model.V) <= model.C_i[i]
#model.charger_capacity_constraint = Constraint(model.T, rule=charger_capacity_rule)


def queuing_system_rule_aux(model, i):
    return model.q_aux[i] == sum(model.x[i, j, k] for j in model.N for k in model.V) - model.C_i[i] + model.s[i]
#model.queuing_system_aux_constraint = Constraint(model.T, rule=queuing_system_rule_aux)

# Constraint 34: Waiting time calculation
def waiting_time_rule(model, k, i):
    return model.W_k[k] == (model.q_i[i] / model.C_i[i]) * model.t_charge
#model.waiting_time_constraint = Constraint(model.V, model.T, rule=waiting_time_rule)

# Constraint 35: Updated time constraints with waiting time
def updated_time_constraints_rule(model, k, i):
    return model.d[i, k] == model.a[i, k] + model.W_k[k] + model.t_charge
#model.updated_time_constraints = Constraint(model.V, model.T, rule=updated_time_constraints_rule)

# Constraint 36: Time windows adjustment (split into two constraints)
def time_windows_lower_bound_rule(model, k, i):
    return model.E[i] <= model.a[i, k] + model.W_k[k]
#model.time_windows_lower_bound_constraint = Constraint(model.V, model.T, rule=time_windows_lower_bound_rule)


# Constraint 37: Relaxed transfer constraints with slack variable
def transfer_constraints_rule(model, k1, k2, i):
    for r in model.R:
        return model.a[i, k2] >= model.d[i, k1] + model.W_k[k1] - model.slack_transfer[k1, k2, i] - model.M * (1 - model.z[i, k1, k2, r])
    return Constraint.Skip

#model.transfer_constraints = Constraint(model.V, model.V, model.T, rule=transfer_constraints_rule)
# Constraint 38: Changes to the objective function
def objective_with_waiting_cost_rule(model):
    return model.objective + sum(model.W_k[k] for k in model.V)
#model.objective_with_waiting_cost = Objective(rule=objective_with_waiting_cost_rule, sense=minimize)

# Deactivate the first objective
#model.objective.deactivate()

# Activate the second objective
#model.objective_with_waiting_cost.activate()
# ====================== SOLVE THE MODEL ======================
# Run config
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
solver.options['mipgap'] = 0.1
solver.options['tmlim'] = 300
results = solver.solve(model, tee=True)

# Log infeasible constraints
if results.solver.termination_condition == TerminationCondition.infeasible:
    print("ُSolution is infeasible duo to : ")
    log_infeasible_constraints(model)
    log_infeasible_bounds(model)

# Print results
print("\n\n ======== RESULTS =======")
print("\nDemands : ")
for r in model.R:
    print(f"commodity = {r}: {L[pickup_nodes[r]]}")

    if results.solver.termination_condition == TerminationCondition.optimal:
        print("\nYij values: ")
        for i in model.N:
            for j in model.N:
                if i != j:
                    for k in model.V:
                        if model.x[i, j, k].value is not None and model.x[i, j, k].value > 0:
                            print(f"arc = ({i}, {j}), vehicle = {k}: {model.x[i, j, k].value}")

        print("\nXijk values: ")
        for i in model.N:
            for j in model.N:
                if i != j:
                    for k in model.V:
                        for r in model.R:
                            if model.y[i, j, k, r].value is not None and model.y[i, j, k, r].value > 0:
                                print(f"arc = ({i}, {j}), vehicle = {k}, commodity = {r}: {model.y[i, j, k, r].value}")
    else:
        print("Solver did not find a feasible solution.")

# ====================== HYBRID HEURISTIC ======================
# ====================== HEURISTIC ALGORITHM ======================
class Solution:
    def __init__(self, routes):
        self.routes = routes
        self.cost = self.calculate_cost()

    def calculate_cost(self):
        total_cost = 0
        for k in model.V:
            route = self.routes[k]
            for i in range(len(route) - 1):
                total_cost += model.c[route[i], route[i + 1]]
        return max(total_cost, 10)

def hybrid_heuristic(S, t, c, MI):
    i = 0  # Non-improved iteration counter
    t_hat = t  # Initial temperature
    rho = 1 / 8  # Probability for neighborhood selection
    S_c = S  # Current solution
    obj_c = S.cost  # Current objective value
    obj_star = S.cost  # Best objective value
    S_star = S  # Best solution

    while True:
        # Select a neighborhood based on roulette wheel selection
        sigma = random.uniform(0, 1)
        if sigma <= rho:
            S_n = P_Swap_Single(S_c)
        elif sigma <= 2 * rho:
            S_n = D_Swap_Single(S_c)
        elif sigma <= 3 * rho:
            S_n = MergeDP(S_c)
        elif sigma <= 4 * rho:
            S_n = P_Swap_Two(S_c)
        elif sigma <= 5 * rho:
            S_n = D_Swap_Two(S_c)
        elif sigma <= 6 * rho:
            S_n = Small_Destroy_Repair(S_c)
        elif sigma <= 7 * rho:
            S_n = T_Insert(S_c)
        else:
            S_n = T_Update(S_c)

        # Check feasibility of the new solution
        if is_feasible(S_n) and S_n.cost > 0:
            obj_n = S_n.cost
            if obj_n < obj_c:
                # Accept the new solution if it improves the objective
                S_c = S_n
                obj_c = obj_n
                if obj_n < obj_star:
                    # Update the best solution
                    S_star = S_n
                    obj_star = obj_n
            else:
                # Accept the new solution with a probability (Simulated Annealing)
                alpha = random.uniform(0, 1)
                if alpha <= math.exp(-(obj_n - obj_c) / t):
                    S_c = S_n
                    obj_c = obj_n
            i += 1
        else:
            # Reject infeasible solutions
            i += 1

        # Update temperature
        t = t * c
        if t < t_hat:
            t = t_hat  # Reset temperature to initial value

        # Stop if the maximum number of non-improved iterations is reached
        if i == MI:
            return S_star

# ====================== NEIGHBORHOOD STRUCTURES ======================
def P_Swap_Single(S):
    routes = S.routes.copy()
    k = random.choice(list(model.V))
    route = routes[k]

    # Identify pickup nodes in the route
    pickup_nodes_in_route = [i for i in route if model.L[i] > 0]

    if len(pickup_nodes_in_route) >= 2:
        i, j = random.sample(pickup_nodes_in_route, 2)  # Select two pickup nodes to swap
        idx_i, idx_j = route.index(i), route.index(j)

        # Get the corresponding delivery nodes
        delivery_i = delivery_nodes[next(r for r in model.R if pickup_nodes[r] == i)]
        delivery_j = delivery_nodes[next(r for r in model.R if pickup_nodes[r] == j)]

        # Ensure delivery nodes are in the route
        if delivery_i not in route or delivery_j not in route:
            return S  # Skip the swap if delivery nodes are not in the route

        # Swap pickup nodes
        route[idx_i], route[idx_j] = route[idx_j], route[idx_i]

        # Ensure delivery nodes are still after their pickup nodes
        if route.index(delivery_i) < route.index(i):
            route.remove(delivery_i)
            route.insert(route.index(i) + 1, delivery_i)
        if route.index(delivery_j) < route.index(j):
            route.remove(delivery_j)
            route.insert(route.index(j) + 1, delivery_j)

    return Solution(routes)

def D_Swap_Single(S):
    routes = S.routes.copy()
    k = random.choice(list(model.V))
    route = routes[k]

    # Identify delivery nodes in the route
    delivery_nodes_in_route = [i for i in route if model.L[i] < 0]

    if len(delivery_nodes_in_route) >= 2:
        i, j = random.sample(delivery_nodes_in_route, 2)  # Select two delivery nodes to swap
        idx_i, idx_j = route.index(i), route.index(j)

        # Get the corresponding pickup nodes
        pickup_i = pickup_nodes[next(r for r in model.R if delivery_nodes[r] == i)]
        pickup_j = pickup_nodes[next(r for r in model.R if delivery_nodes[r] == j)]

        # Ensure pickup nodes are in the route and before their delivery nodes
        if pickup_i not in route or pickup_j not in route:
            return S  # Skip the swap if pickup nodes are not in the route

        # Swap delivery nodes
        route[idx_i], route[idx_j] = route[idx_j], route[idx_i]

        # Ensure pickup nodes are still before their delivery nodes
        if route.index(pickup_i) > route.index(i):
            route.remove(pickup_i)
            route.insert(route.index(i) - 1, pickup_i)
        if route.index(pickup_j) > route.index(j):
            route.remove(pickup_j)
            route.insert(route.index(j) - 1, pickup_j)

    return Solution(routes)

def MergeDP(S):
    """
    Merge pickup and delivery nodes into the same route.
    """
    routes = S.routes.copy()
    k = random.choice(list(model.V))
    route = routes[k]
    # Find a request where pickup and delivery are in different routes
    for r in model.R:
        p_r = pickup_nodes[r]  # Pickup node of request r
        d_r = delivery_nodes[r]  # Delivery node of request r
        if p_r in route and d_r not in route:
            # Insert delivery node after pickup node
            idx_p = route.index(p_r)
            route.insert(idx_p + 1, d_r)
    return Solution(routes)

def P_Swap_Two(S):
    """
    Swap two pickup nodes between routes, ensuring that the delivery nodes are still visited after their corresponding pickup nodes.
    """
    routes = S.routes.copy()
    k1, k2 = random.sample(list(model.V), 2)  # Select two random vehicles
    route1, route2 = routes[k1], routes[k2]

    # Identify pickup nodes in the routes
    pickup_nodes1 = [i for i in route1 if model.L[i] > 0]
    pickup_nodes2 = [i for i in route2 if model.L[i] > 0]

    if pickup_nodes1 and pickup_nodes2:
        i, j = random.choice(pickup_nodes1), random.choice(pickup_nodes2)  # Select two pickup nodes to swap
        idx_i, idx_j = route1.index(i), route2.index(j)

        # Get the corresponding delivery nodes
        delivery_i = delivery_nodes[next(r for r in model.R if pickup_nodes[r] == i)]
        delivery_j = delivery_nodes[next(r for r in model.R if pickup_nodes[r] == j)]

        # Ensure pickup nodes and delivery nodes are in the same routes
        if delivery_i not in route1 or delivery_j not in route2:
            return S  # Skip the swap if delivery nodes are not in the same routes

        # Swap pickup nodes
        route1[idx_i], route2[idx_j] = route2[idx_j], route1[idx_i]

        # Ensure delivery nodes are still after their pickup nodes
        if delivery_i in route1 and i in route1 and route1.index(delivery_i) < route1.index(i):
            route1.remove(delivery_i)
            route1.insert(route1.index(i) + 1, delivery_i)
        if delivery_j in route2 and j in route2 and route2.index(delivery_j) < route2.index(j):
            route2.remove(delivery_j)
            route2.insert(route2.index(j) + 1, delivery_j)

    return Solution(routes)

def D_Swap_Two(S):
    """
    Swap two delivery nodes between routes, ensuring that the pickup nodes are still visited before their corresponding delivery nodes.
    """
    routes = S.routes.copy()
    k1, k2 = random.sample(list(model.V), 2)  # Select two random vehicles
    route1, route2 = routes[k1], routes[k2]

    # Identify delivery nodes in the routes
    delivery_nodes1 = [i for i in route1 if model.L[i] < 0]
    delivery_nodes2 = [i for i in route2 if model.L[i] < 0]

    if delivery_nodes1 and delivery_nodes2:
        i, j = random.choice(delivery_nodes1), random.choice(delivery_nodes2)  # Select two delivery nodes to swap
        idx_i, idx_j = route1.index(i), route2.index(j)

        # Get the corresponding pickup nodes
        pickup_i = pickup_nodes[next(r for r in model.R if delivery_nodes[r] == i)]
        pickup_j = pickup_nodes[next(r for r in model.R if delivery_nodes[r] == j)]

        # Ensure pickup nodes are in the same routes as their delivery nodes
        if pickup_i not in route1 or pickup_j not in route2:
            return S  # Skip the swap if pickup nodes are not in the same routes

        # Swap delivery nodes
        route1[idx_i], route2[idx_j] = route2[idx_j], route1[idx_i]

        # Ensure pickup nodes are still before their delivery nodes
        if pickup_i in route1 and i in route1 and route1.index(pickup_i) > route1.index(i):
            route1.remove(pickup_i)
            route1.insert(route1.index(i) - 1, pickup_i)
        if pickup_j in route2 and j in route2 and route2.index(pickup_j) > route2.index(j):
            route2.remove(pickup_j)
            route2.insert(route2.index(j) - 1, pickup_j)

    return Solution(routes)
def Small_Destroy_Repair(S):
    """
    Remove and reinsert a request, ensuring that the pickup node is visited before the delivery node.
    """
    routes = S.routes.copy()
    r = random.choice(list(model.R))
    p_r, d_r = pickup_nodes[r], delivery_nodes[r]

    for k in model.V:
        if p_r in routes[k]:
            routes[k].remove(p_r)
        if d_r in routes[k]:
            routes[k].remove(d_r)
    # Reinsert the request into the best position
    best_cost = float('inf')
    best_routes = routes
    for k in model.V:
        new_routes = routes.copy()
        new_routes[k].append(p_r)
        new_routes[k].append(d_r)
        cost = Solution(new_routes).cost
        if cost < best_cost:
            best_cost = cost
            best_routes = new_routes
    return Solution(best_routes)

def T_Insert(S):
    routes = S.routes.copy()
    t_node = random.choice(list(model.T))  # Select a random transfer node
    k1, k2 = random.sample(list(model.V), 2)  # Select two random vehicles

    # Insert transfer node into routes while respecting battery constraints
    if t_node not in routes[k1]:
        routes[k1].append(t_node)
    if t_node not in routes[k2]:
        routes[k2].append(t_node)

    return Solution(routes)
def T_Update(S):
    """
    Update transfer nodes in routes, ensuring that the pickup and delivery nodes are still visited in the correct order.
    """
    routes = S.routes.copy()
    t_node = random.choice(list(model.T))  # Select a random transfer node
    for k in model.V:
        if t_node in routes[k]:
            routes[k].remove(t_node)
    return Solution(routes)

# ====================== FEASIBILITY CHECK ======================
# ====================== FEASIBILITY CHECK ======================
def is_feasible(S):
    # Check vehicle capacity constraints
    for k in model.V:
        load = 0
        for i in S.routes[k]:
            load += model.L[i]
        if load > model.Q[k]:
       #     print(f"Vehicle {k} exceeds capacity: load = {load}, capacity = {model.Q[k]}")
            return False

    # Check battery constraints
    for k in model.V:
        battery = model.B[k]
        for i in range(len(S.routes[k]) - 1):
            current_node = S.routes[k][i]
            next_node = S.routes[k][i + 1]
            battery -= model.t[current_node, next_node]
            if battery < 0:
       #         print(f"Vehicle {k} runs out of battery between {current_node} and {next_node}")
                return False
            if next_node in model.T:
                battery = min(battery + model.H[k], model.B[k])  # Recharge at transfer nodes

    # Check pickup and delivery precedence
    for r in model.R:
        pickup_node = pickup_nodes[r]
        delivery_node = delivery_nodes[r]
        for k in model.V:
            route = S.routes[k]
            if pickup_node in route and delivery_node in route:
                pickup_index = route.index(pickup_node)
                delivery_index = route.index(delivery_node)
                if delivery_index <= pickup_index:
                 #   print(f"Precedence violated for request {r} in vehicle {k}: "
                 #         f"pickup at {pickup_index}, delivery at {delivery_index}")
                    return False  # Precedence violated

    # Check time window constraints
    for k in model.V:
        current_time = 0
        for i in range(len(S.routes[k]) - 1):
            current_node = S.routes[k][i]
            next_node = S.routes[k][i + 1]
            current_time += model.t[current_node, next_node]
            if next_node in model.T:
                current_time += model.p[next_node, k]  # Add charging time at transfer nodes
            if next_node in model.D and current_time > model.T_i[next_node]:
            #    print(f"Vehicle {k} violates time window at node {next_node}: "
            #          f"arrival time = {current_time}, latest time = {model.T_i[next_node]}")
                return False

#   print("Solution is feasible.")
    return True

# ====================== INITIAL SOLUTION ======================
def generate_initial_solution():
    routes = {k: [] for k in model.V}  # Initialize routes for each vehicle

    for r in model.R:
        k = random.choice(list(model.V))  # Assign request to a random vehicle
        pickup_node = pickup_nodes[r]  # Get the pickup node for the request
        delivery_node = delivery_nodes[r]  # Get the delivery node for the request

        # Insert pickup and delivery nodes in the correct order
        if routes[k]:
            # If route is not empty, insert pickup and delivery at appropriate positions
            routes[k].append(pickup_node)
            routes[k].append(delivery_node)
        else:
            # If route is empty, add pickup and delivery nodes
            routes[k] = [pickup_node, delivery_node]

    return Solution(routes)

# ====================== RUN THE MODEL ======================
# Generate initial solution
initial_solution = generate_initial_solution()

# Check feasibility of the initial solution
if is_feasible(initial_solution):
    print("Initial solution is feasible.")
    print("Initial solution routes:", initial_solution.routes)
else:
    print("Initial solution is infeasible. Regenerating...")
    # Regenerate initial solution until it is feasible
    while not is_feasible(initial_solution):
        initial_solution = generate_initial_solution()
    print("Feasible initial solution found.")
    print("Initial solution routes:", initial_solution.routes)

# Run the heuristic
best_solution = hybrid_heuristic(initial_solution, 100000, 40, 1000000)

# Check feasibility of the best solution
if is_feasible(best_solution):
    print("Best Solution is Feasible")
    print("Best Solution Cost:", best_solution.cost)
    print("Best Solution Routes:", best_solution.routes)
else:
    print("Best Solution is Infeasible")



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/usr/bin/glpsol
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --mipgap 0.1 --tmlim 300 --write /tmp/tmpwtlovone.glpk.raw --wglp /tmp/tmp_waf7r5i.glpk.glp
 --cpxlp /tmp/tmp8vhq7k8l.pyomo.lp
Reading problem data from '/tmp/tmp8vhq7k8l.pyomo.lp'...
/tmp/tmp8vhq7k8l.pyomo.lp:7261: warning: lower bound of variable 'x1415' redefined
/tmp/tmp8vhq7k8l.pyomo.lp:7261: warning: upper bound of variable 'x1415' redefined
1127 rows, 659 columns, 3182 non-zeros
600 integer variables, all of which are binary
7861 lines were read
Writing problem data to '/tmp/tmp_waf7r5i.glpk.glp'...
6169 lines were written
GLPK Integer Optimizer 5.0
1127 rows, 659 columns, 3182 non-zeros
600 integer variables, all of which are binary
Preprocessing...
6 hidden packing inequaliti(es)